![An interactive LADAL notebook](https://slcladal.github.io/images/uq1.jpg)

***

Please copy this Jupyter notebook so that you are able to edit it.

Simply go to: File > Save a copy in Drive.

Once you have done that, you are good to go.

***

This tutorial is the interactive Jupyter notebook accompanying the [Language Technology and Data Analysis Laboratory (LADAL) tutorial *Network Analysis using R*](https://ladal.edu.au/net.html). 



***


**Preparation and session set up**

If you are using this notebook on Google Colab or your own computer and you have not already installed the R packages listed below, you need to install them. You can install them by running the code chunk below. 


In [ ]:
# install packages
install.packages("GGally")
install.packages("ggraph")
install.packages("igraph")
install.packages("Matrix")
install.packages("network")
install.packages("quanteda")
install.packages("sna")
install.packages("tidygraph")
install.packages("tidyverse")
install.packages("tm")
install.packages("tibble")
install.packages("quanteda.textplots")


***



In [ ]:
# activate packages
library(GGally)
library(ggraph)
library(igraph)
library(Matrix)
library(network)
library(quanteda)
library(sna)
library(tidygraph)
library(dplyr)
library(tidyr)
library(tm)
library(tibble)


Once you have initiated the session by executing the code shown above, you are good to go.

***


# Data preparation



This section shows how to create a network visualization with the `network` and the `GGally` packages. The network we will generate shows how often characters in William Shakespeare's *Romeo and Juliet* occurred in the same scene. The issue we want to investigate here relates to networks of personas in Shakespeare's *Romeo and Juliet* and we thus load this famous work of fiction. 


## Loading data

Now that we have loaded the data, we need to split the data into scenes. Scenes during which personas leave or enter will have to be split too so that we arrive at a table that contains the personas that are present during a sub-scene.


In [ ]:
# load data
rom <- read.delim("https://slcladal.github.io/data/romeo_tidy.txt", sep = "\t")
# inspect data
rom %>%
  as.data.frame() %>%
  head(15)


## Using your own data

While the tutorial uses data from the LADAL website, you can also use your own data. You can see below what you need to do to upload and use your own data.

The code chunk below allows you to upload two files from your own computer. To be able to load your own data, you need to click on the folder symbol to the left of the screen:

![Colab Folder Symbol](https://slcladal.github.io/images/ColabFolder.png)

Then on the upload symbol.

![Colab Upload Symbol](https://slcladal.github.io/images/ColabUpload.png)

Next, upload the files you want to analyze and then the respective files names in the file argument of the scan function. When you then execute the code (like to code chunk below, you will upload your own data.


In [ ]:
mytext1 <- scan(file = "linguistics01.txt",
            what = "char", 
            sep = "", 
            quote = "", 
            quiet = T, 
            skipNul = T) %>%
            paste0(collapse = " ")
mytext2 <- scan(file = "linguistics02.txt",
            what = "char", 
            sep = "", 
            quote = "", 
            quiet = T, 
            skipNul = T) %>%
            paste0(collapse = " ")
# inspect
mytext1; mytext2


**Keep in mind though that you need to adapt the names of the texts in the code chunks below so that the code below work on your own texts!**

***

## Creating a matrix

We now transform that table into a co-occurrence matrix.


In [ ]:
rome <- crossprod(table(rom[1:2]))
diag(rome) <- 0
romeo <- as.data.frame(rome)
# inspect data
romeo[1:5, 1:5]%>%
  as.data.frame() %>%
  tibble::rownames_to_column("Persona")


The data shows how often a character has appeared with each other character in the play - only Friar Lawrence and Friar John were excluded because they only appear in one scene where they talk to each other. 

# Network Visualization

There are various different ways to visualize a network structure. We will focus on two packages for network visualization here and exemplify how you can visualize networks in R. 

## Tidy Networks

A great way to generate network graphs is to combine functions from the `igraph`, the `ggraph`, and the `tidygraph` packages. The advantages are that the syntax of for creating the networks aligns with the tidyverse style of writing R and that the graphs can be modified very easily.

To generate network graphs in this way, we define the *nodes* and we can also add information about the odes that we can use later on (such as frequency information).


In [ ]:
va <- romeo %>%
  dplyr::mutate(Persona = rownames(.),
                Occurrences = rowSums(.)) %>%
  dplyr::select(Persona, Occurrences) %>%
  dplyr::filter(!str_detect(Persona, "SCENE"))
va %>%
  as.data.frame()


Now, we define the *edges*, i.e., the connections between nodes and, again, we can add information in separate variables that we can use later on. 



In [ ]:
ed <- romeo %>%
  dplyr::mutate(from = rownames(.)) %>%
  tidyr::gather(to, Frequency, BALTHASAR:TYBALT) %>%
  dplyr::mutate(Frequency = ifelse(Frequency == 0, NA, Frequency))
ed %>%
  as.data.frame() %>%
  head(15)


Now that we have generated tables for the edges and the nodes, we can generate a graph object.



In [ ]:
ig <- igraph::graph_from_data_frame(d=ed, vertices=va, directed = FALSE)



We will also add labels to the nodes as follows:



In [ ]:
tg <- tidygraph::as_tbl_graph(ig) %>% 
  tidygraph::activate(nodes) %>% 
  dplyr::mutate(label=name)


When we now plot our network, it looks as shown below.



In [ ]:
# set seed
set.seed(12345)
# edge size shows frequency of co-occurrence
tg %>%
   ggraph(layout = "fr") +
   geom_edge_arc(colour= "gray50",
                 lineend = "round",
                 strength = .1,
                 alpha = .1) +
   geom_node_text(aes(label = name), 
                  repel = TRUE, 
                  point.padding = unit(0.2, "lines"), 
                  colour="gray10") +
  theme_graph(background = "white") +
  guides(edge_width = FALSE,
         edge_alpha = FALSE)


Now, we use the number of occurrences to define vertex size (or node size): the more often a character appears, the bigger it will appear in the graph.



In [ ]:
v.size <- V(tg)$Occurrences
# inspect
v.size


When we include this into our network, it looks as shown below.



In [ ]:
# set seed
set.seed(12345)
# edge size shows frequency of co-occurrence
tg %>%
   ggraph(layout = "fr") +
   geom_edge_arc(colour= "gray50",
                  lineend = "round",
                 strength = .1) +
   geom_node_point(size=log(v.size)*2) +
   geom_node_text(aes(label = name), 
                  repel = TRUE, 
                  point.padding = unit(0.2, "lines"), 
                  size=sqrt(v.size), 
                  colour="gray10") +
  scale_edge_width(range = c(0, 2.5)) +
  scale_edge_alpha(range = c(0, .3)) +
  theme_graph(background = "white") +
  guides(edge_width = FALSE,
         edge_alpha = FALSE)


Next, we modify the edges by using frequency information to define weights: the more often two characters appear in the same scene, the bigger the edge.



In [ ]:
E(tg)$weight <- E(tg)$Frequency
# inspect weights
head(E(tg)$weight, 10)


When we include this into our network, it looks as shown below.



In [ ]:
# set seed
set.seed(12345)
# edge size shows frequency of co-occurrence
tg %>%
   ggraph(layout = "fr") +
   geom_edge_arc(colour= "gray50",
                  lineend = "round",
                 strength = .1,
                 aes(edge_width = weight,
                     alpha = weight)) +
   geom_node_point(size=log(v.size)*2) +
   geom_node_text(aes(label = name), 
                  repel = TRUE, 
                  point.padding = unit(0.2, "lines"), 
                  size=sqrt(v.size), 
                  colour="gray10") +
  scale_edge_width(range = c(0, 2.5)) +
  scale_edge_alpha(range = c(0, .3)) +
  theme_graph(background = "white") +
  theme(legend.position = "top") +
  guides(edge_width = FALSE,
         edge_alpha = FALSE)


Finally, we define colors so that characters belonging to the same family have the same color.



In [ ]:
# define colors (by family)
mon <- c("ABRAM", "BALTHASAR", "BENVOLIO", "LADY MONTAGUE", "MONTAGUE", "ROMEO")
cap <- c("CAPULET", "CAPULET’S COUSIN", "FIRST SERVANT", "GREGORY", "JULIET", "LADY CAPULET", "NURSE", "PETER", "SAMPSON", "TYBALT")
oth <- c("APOTHECARY", "CHORUS", "FIRST CITIZEN", "FIRST MUSICIAN", "FIRST WATCH", "FRIAR JOHN" , "FRIAR LAWRENCE", "MERCUTIO", "PAGE", "PARIS", "PRINCE", "SECOND MUSICIAN", "SECOND SERVANT", "SECOND WATCH", "SERVANT", "THIRD MUSICIAN")
# create color vectors
Family <- dplyr::case_when(sapply(tg, "[")$nodes$name %in% mon ~ "MONTAGUE",
                           sapply(tg, "[")$nodes$name %in% cap ~ "CAPULET",
                           TRUE ~ "Other")
# inspect colors
Family


Now, that we have created the different objects and defined their properties, we can finally visualize the finished network.



In [ ]:
# set seed
set.seed(12345)
# edge size shows frequency of co-occurrence
tg %>%
   ggraph(layout = "fr") +
   geom_edge_arc(colour= "gray50",
                  lineend = "round",
                 strength = .1,
                 aes(edge_width = weight,
                     alpha = weight)) +
   geom_node_point(size=log(v.size)*2, 
                   aes(color=Family)) +
   geom_node_text(aes(label = name), 
                  repel = TRUE, 
                  point.padding = unit(0.2, "lines"), 
                  size=sqrt(v.size), 
                  colour="gray10") +
  scale_edge_width(range = c(0, 2.5)) +
  scale_edge_alpha(range = c(0, .3)) +
  theme_graph(background = "white") +
  theme(legend.position = "top") +
  guides(edge_width = FALSE,
         edge_alpha = FALSE)


## Quanteda Networks 

The `quanteda` package contains many very useful functions for analyzing texts. Among these functions is the `textplot_network` function which provides a very handy way to display networks. The advantage of the network plots provided by or generated with the `quanteda` package is that you can create them with very little code. However, this comes at a cost as  these visualizations cannot be modified easily (which means that their design is not very flexible compared to other methods for generating network visualizations).    

In a first step, we transform the text vectors of the `romeo` data into a document-feature matrix using the `dfm ` function. 


In [ ]:
# create a document feature matrix
romeo_dfm <- quanteda::as.dfm(romeo)
# create feature co-occurrence matrix
romeo_fcm <- quanteda::fcm(romeo_dfm)
# inspect data
head(romeo_fcm)


This feature-co-occurrence matrix can then serve as the input for the `textplot_network` function which already generates a nice network graph. The network graph can then be modified or customized easily by defining the arguments of the `textplot_network` function. To see how and which arguments can be modified, you can use `?textplot_network`.



In [ ]:
quanteda.textplots::textplot_network(romeo_fcm, 
                                     min_freq = .5, 
                                     edge_alpha = 0.5, 
                                     edge_color = "purple",
                                     vertex_labelsize = log(rowSums(romeo_fcm)),
                                     edge_size = 2)


# Network Statistics

In addition to visualizing networks, we will analyze the network and extract certain statistics about the network that tell us about structural properties of networks. In the present case, we will extract 

To extract the statistics, we use the edge object generated above (called `ed`) and remove all rows that contain missing values (`NA`) and then repeat each combination as often as it occurred based on the value in the `Frequency` column.


In [ ]:
dg <- ed %>%
  tidyr::drop_na()
dg <- dg[rep(seq_along(dg$Frequency), dg$Frequency), 1:2]
rownames(dg) <- NULL
# inspect data
dg %>%
  as.data.frame() %>%
  head(15)


We now generate an edge list from the `dg` object and then extract the *degree centrality* of all edges in the `dg` object by using the `degree` function from the `igraph` package. The *degree centrality* reflects the counts of how many edges each node has. The most central node is the one with with the highest value of ties.



In [ ]:
dgg <- graph.edgelist(as.matrix(dg), directed = T)
# extract degree centrality
igraph::degree(dgg) 


To extract the most central node, we can use the `max()` function to extract the name of the node with the most edges.



In [ ]:
names(igraph::degree(dgg))[which(igraph::degree(dgg) == max(igraph::degree(dgg)))]



We now  extract the *betweenness centrality* of all edges in the `dg` object by using the `betweenness` function from the `igraph` package. *Betweenness centrality* reflects how connected nodes are. More precisely, it provides a measure of how important noes are for information flow between nodes in a  network. The node with the highest betweenness centrality creates the shortest  paths in the network. For any two nodes we can find the shortest path, i.e. the path with the lowest number of steps that are necessary to get from the node A to node B. If a node C facilitates the shortest path between A and B, then it means C is important to the efficient flow of information between A and B. 

Thus, betweenness effectively counts how many shortest paths each node is on. The higher a node’s betweenness, the more important they are for the efficient flow of goods in a network.


In [ ]:
igraph::betweenness(dgg)



To extract the most central node, we can use the `max()` function to extract the name of the node with the most edges.



In [ ]:
names(igraph::betweenness(dgg))[which(igraph::betweenness(dgg) == max(igraph::betweenness(dgg)))]



In addition, we extract the closeness statistic of all edges in the `dg` object by using the `closeness` function from the `igraph` package. Closeness centrality refers to the shortest paths between nodes. The distance between two nodes represents the length of the shortest path between them. The closeness of a node is the average distance from that node to all other nodes. 



In [ ]:
igraph::closeness(dgg)



To extract the node with the highest closeness, we can use the `max()` function to extract the name of the node with the most edges.



In [ ]:
names(igraph::closeness(dgg))[which(igraph::closeness(dgg) == max(igraph::closeness(dgg)))]



We have reached the end of this tutorial and you now know how to create and modify networks in R and how you can highlight aspects of your data. 


***

[Back to LADAL](https://ladal.edu.au/net.html)

***
